In [ ]:
#general packages
import pandas as pd
import numpy as np
from collections import Counter
import tifffile as tf
from skimage.measure import regionprops
#plotting packages
import matplotlib.pyplot as plt
import seaborn as sns
#custom function
from post_analysis import *
%config InlineBackend.figure_format='retina'

In [ ]:
#for across channel
mtx = pd.read_csv(f"/groups/CaiLab/personal/Lex/raw/230608_4k_inv_5bs/pyfish_tools/output/genebycell/final_0.7511.25_seed44_heg_svm_p20.0_diff1_fdr10.0/final/genebycell.csv", index_col=0)

In [ ]:
#take a look
mtx

In [ ]:
#codebook
codebook = pd.read_csv(f"/groups/CaiLab/personal/Lex/raw/230608_4k_inv_5bs/barcode_key/codebook_string_across.csv", index_col=0)
#separate into true and false codebook
fakebook = codebook[codebook.index.str.startswith("fake")]
codebook = codebook.drop(fakebook.index)

In [ ]:
#calculate fdr
fp, fake = percent_false_positive(mtx, codebook, fakebook)
percent_fp = fp["FP raw"].mean()
mean_counts = fp["total_real"].mean()
sum_counts = fp["total_counts"].sum()
norm_fpr = fp["FDR"].mean()
fp_list = [percent_fp,norm_fpr,mean_counts,sum_counts]

In [ ]:
#take a look at fdr results
df_stats = pd.DataFrame(fp_list).T
df_stats.columns = ["percent fp","false positive rate","mean true counts", "total sum"]
df_stats

# Efficiency and correlations (if applicable)

In [ ]:
#read in rnaseq data
rnaseq = pd.read_csv("./RNAseq_files/NIH3T3_CCS_TPM_REP1.csv")
rnaseq.columns = ["Genes","TPM"]

In [ ]:
#convert data to pseudobulk rnaseq data
bulk = pd.DataFrame(mtx.mean(axis=1)).reset_index()
bulk.columns = ["Genes", "Counts"]
bulk["Genes"] = bulk["Genes"].str.lower()
rnaseq["Genes"] = rnaseq["Genes"].str.lower()
#merge
comb_1 = pd.merge(rnaseq,bulk)
#pearson's correlation
r = pearsonr(comb_1["TPM"],comb_1["Counts"])
r = round(r[0],2)

In [ ]:
#get log2 + 1
comb_1["Log Counts"] = np.log10(comb_1["Counts"]+0.1)
comb_1["Log TPM"] = np.log10(comb_1["TPM"]+0.1)

In [ ]:
#RNA-seq plot
sns.set_style("white")
joint_kws=dict(gridsize=50)
hexplot = sns.jointplot(data=comb_1, x="Log TPM", y="Log Counts", kind="hex",mincnt=0.1, 
              cmap="plasma", dropna=True, joint_kws=joint_kws)
plt.xlabel("Bulk RNAseq Log10(TPM+0.1)", fontsize=12)
plt.ylabel("Pseudobulk Log10(Counts+0.1)", fontsize=12)
hexplot.ax_marg_x.remove()
hexplot.ax_marg_y.remove()
plt.annotate(f"Pearson's r= {r}", (-1.0,0.4), fontsize=12)
plt.title("All Channels", fontweight="bold")
plt.colorbar()
sns.despine()
plt.show()

In [ ]:
#read in smfish and other reference files
smfish_density = pd.read_csv("./nih3t3_smfish/27gene_smfish_density.csv", index_col=0)
_150genes_density = pd.read_csv("./nih3t3_smfish/150_genes_density.csv", index_col=0)
mtx_den = pd.read_csv(f"/groups/CaiLab/personal/Lex/raw/230608_4k_inv_5bs/pyfish_tools/output/genebycell/final_0.7511.25_seed33_heg_svm_p20.0_diff1_fdr10.0/final/gene_density_all.csv", index_col=0)

In [ ]:
correlation(mtx_den,smfish_density, label_x="smFISH", label_y="LANTERN",
            title="All Channels", cell_size_normalized=True, 
            return_comb_df=False, log=False)

In [ ]:
#150 gene density correlation
correlation(mtx_den,_150genes_density, label_x="150 genes", label_y="LANTERN",
            title="All Channels", cell_size_normalized=True, 
            return_comb_df=False, log=False)

# Percent decoded

In [ ]:
#get average percent decoded
percent_decoded_list = []
for i in range(52):
    for z in range(1):
        try:
            src = f""
            with open(src) as f:
                decoded = f.readlines()[0].split(" ")[-1]
                f.close()
                percent_decoded_list.append(float(decoded))
        except FileNotFoundError:
            continue

In [ ]:
np.mean(percent_decoded_list)

# Single-Cell Analysis

In [ ]:
#analysis packages
import pandas as pd
import numpy as np
import scanpy as sc
import anndata
import scrublet as scr
from sklearn.linear_model import LinearRegression
import scipy.stats as st
import gseapy as gp

#plotting packages
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rcParams

%config InlineBackend.figure_format = 'retina'

According to Booeshaghi et al,  proportional fitting prior to log transformation followed by an additional proportional fitting is best for normalization (ref 1). 

ref 1: https://www.biorxiv.org/content/10.1101/2022.05.06.490859v1

Another source from Ahlmann-Eltze, says that the logarithm of a pseudo-count followed by principal-component analysis, performs as well or better than the more sophisticated alternatives

ref 2: https://www.nature.com/articles/s41592-023-01814-1

Do to these reasons, I'm just going to proceed with PFLogPF for normalization instead of sctransform or LogCP10k.

In [ ]:
#--------------------------------------------
import scipy as sp
#from ref 1
def do_pf(mtx, sf = None):
    pf = mtx.sum(axis=1).ravel()
    if not sf:
        sf = pf.mean()
    pf = sp.sparse.diags(sf/pf) @ mtx
    return pf

def norm_pf_log_pf(mtx):
    pf_log_pf = do_pf(np.log1p(do_pf(mtx)))
    return pf_log_pf
#--------------------------------------------

In [ ]:
mtx = mtx[~mtx.index.str.startswith("fake")]

In [ ]:
adata = sc.AnnData(mtx.T, dtype=mtx.values.dtype)

In [ ]:
#This filters cells based on minimal UMI counts. 
sc.pp.filter_cells(adata, min_counts = 5)

In [ ]:
#This filters genes based on minimal number of counts
sc.pp.filter_genes(adata, min_counts = 5)

In [ ]:
np.median(adata.var['n_counts'])

In [ ]:
adata.X = norm_pf_log_pf(adata.X.T).T

In [ ]:
adata

In [ ]:
#find highly variable genes
sc.pp.highly_variable_genes(adata, min_mean=0.01, max_mean=8,
                            min_disp=1, n_top_genes=10, n_bins=20, flavor="seurat")
sc.pl.highly_variable_genes(adata)

In [ ]:
#mean center data and divide by std
sc.pp.scale(adata, max_value=10)

In [ ]:
#perform UMAP
sc.tl.pca(adata, svd_solver='arpack', use_highly_variable=True, n_comps=20)
sc.pp.neighbors(adata, n_neighbors=30, n_pcs=20, knn=True)
sc.tl.umap(adata)

In [ ]:
#unsupervised clustering with leiden
sc.tl.leiden(adata, resolution = 0.5)

In [ ]:
adata.var

In [ ]:
#visualize plot
sc.pl.umap(adata, color="slc17a7", title = "", show = False)
sns.despine()
plt.show()

In [ ]:
#visualize plot
from pylab import rcParams
rcParams['figure.figsize'] = 5, 5

sc.pl.umap(adata, color="leiden", title = "", show = False, alpha=0.5)
sns.despine()
plt.show()

In [ ]:
markers = pd.read_csv("/groups/CaiLab/personal/Lex/raw/230521_10k_human_AD/top25_cluster-markers_2023_0524.csv")

In [ ]:
sc.tl.rank_genes_groups(adata,'leiden', method='wilcoxon')
groups = sc.get.rank_genes_groups_df(adata, None)

In [ ]:
types = markers.cluster.unique()

In [ ]:
markers

In [ ]:
cluster_def = []
for uni in range(len(groups.group.unique())):
    cls = groups[groups.group == str(uni)]
    celltype = []
    for cell in types:
        genes = markers[markers.cluster == cell].gene.values
        genes = [gene.lower() for gene in genes]
        overlap = len(set(cls.names.values) & set(genes))
        celltype.append([cell, overlap/len(genes)])
    celltype = np.array(celltype)
    best = np.argmax(celltype[:,1].astype(float))
    cluster_def.append([uni, celltype[best]])

In [ ]:
cluster_def

# Doublet detection (if masks were too big)

In [ ]:
#perform scrublet modeling 
#use unnormalized counts matrix
scrub = scr.Scrublet(genes_tpt_live_unnorm.X, expected_doublet_rate = 0.08)

In [ ]:
#calculate doublet scores
doublet_scores, predicted_doublets = scrub.scrub_doublets(min_counts=1, 
                                                          min_cells=1,
                                                          min_gene_variability_pctl=85, 
                                                          n_prin_comps=50,
                                                         )

In [ ]:
#add labels normalized matrix
genes_tpt_live.obs["scrublet"] = predicted_doublets

In [ ]:
#color by scrublet doublet
sc.pl.umap(genes_tpt_live, color="scrublet", cmap = "viridis", show = False)
sns.despine()

In [ ]:
#remove doublets
genes_tpt_live = genes_tpt_live[genes_tpt_live.obs["scrublet"] == False]